In [1]:
import numpy as np
import pandas as pd
import psycopg2 as pg
import os
import csv

In [4]:
# loading the database credentials from the .gitignore protected password file
pass_file = "/mnt/data/mvesc/pgpass"
f = open(pass_file, 'r')
with open(pass_file, 'r') as f:
   passinfo = f.read()
passinfo = passinfo.strip().split(':')
f.close()
host_address = passinfo[0]
user_name = passinfo[2]
name_of_database = passinfo[3]
user_password = passinfo[4]
conn = pg.connect(host=host_address, database=name_of_database, user=user_name, password=user_password)

# notebook options
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
# %load_ext rpy2.ipython
# %Rpush data_dir absence_file_names

In [6]:
%%R
r_df <- read.csv(paste0(data_dir, absence_file_names[2]), header = F, stringsAsFactors = F)
str(r_df)


Error in paste0(data_dir, absence_file_names[2]) : 
  object 'data_dir' not found


/home/jgutman/env/lib/python3.4/site-packages/rpy2/rinterface/__init__.py:185: RRuntimeWarning: Error in paste0(data_dir, absence_file_names[2]) : 
  object 'data_dir' not found

  warnings.warn(x, RRuntimeWarning)


In [17]:
data_dir = '/mnt/data/mvesc/PartnerData/'
absence_file_names = os.listdir(data_dir)

In [24]:
absence_file_names
absence_file_names[-6]

['MVESC_DistrictRatings.xlsx',
 'DistrictSchoolIDs.csv',
 'UofChi_MVESC0416.bak',
 'DistrictGrades2006_16',
 '.ipynb_checkpoints',
 'MVESCDisticts0616.bak',
 '.Drakefile',
 'OAAOGT_0616.txt',
 'default_profile',
 'AbsenceDaysDetail',
 'DistrictSchoolIDs.xls',
 'MVESC_Mobility.xlsx',
 'default_profile.example']

'OAAOGT_0616.txt'

In [37]:
df_IDs = pd.read_excel(data_dir + absence_file_names[-3], sheetname = None)
df_mobility = pd.read_excel(data_dir + absence_file_names[-2], sheetname = None)

In [65]:
df_OAAOGT = pd.read_csv(data_dir + absence_file_names[-6], nrows = 1000)

In [38]:
df_ratings = pd.read_excel(data_dir + absence_file_names[0], sheetname = None)

In [51]:
df_IDs.keys()
df_mobility.keys()
df_ratings.keys()

dict_keys(['SchoolIDs'])

dict_keys(['MVESC_Mobility'])

dict_keys(['MVESC1314', 'MVESC1213', 'MVESC1415', 'MVESC1112', 'MVESC1011'])

In [64]:
for key in df_ratings.keys():
    print(df_ratings[key].shape)

(14, 104)
(14, 113)
(14, 119)
(14, 106)
(14, 111)


In [82]:
df_OAAOGT.dtypes

AltID                     int64
MI                       object
DOB                      object
Gender                   object
Ethnicity                object
SSN                     float64
KRAL                    float64
KRAL_PL                  object
Third_Read_PL            object
Third_Read_SS            object
Third_Math_PL            object
Third_Math_SS           float64
Fourth_Read_PL           object
Fourth_Read_SS          float64
Fourth_Math_PL           object
Fourth_Math_SS          float64
Fourth_Write_PL          object
Fourth_Write_SS         float64
Fourth_Ctz_PL            object
Fourth_Ctz_SS           float64
Fourth_Science_PL        object
Fourth_Science_SS       float64
Fifth_Read_PL            object
Fifth_Read_SS            object
Fifth_Math_PL            object
Fifth_Math_SS           float64
Fifth_SocStudies_PL      object
Fifth_SocStudies_SS     float64
Fifth_Science_PL         object
Fifth_Science_SS        float64
                         ...   
Sixth_Wr

In [86]:
df_OAAOGT.AltID.max()

33898

In [77]:
df_OAAOGT.SSN[~df_OAAOGT.SSN.isnull()].shape

(165,)

In [10]:
common_delimeters = set(['\' \'', '\'\t\'', '\',\''])
def sniff(filepath):
   with open(filepath, 'rb') as csvfile:
        dialect = csv.Sniffer().sniff(csvfile.read(2048))
        delimiter = repr(dialect.delimiter)
        if delimiter not in common_delimeters:
            print(filepath,'has uncommon delimiter',delimiter)
        else:
            print(filepath,'has common delimiter',delimiter)
        csvfile.seek(0)
        if csv.Sniffer().has_header(csvfile.read(2048)):
            print(filepath, 'has a header')
            return True
        else:
            print(filepath, 'does not have a header')
            return False

In [11]:
sniff('/mnt/data/mvesc/PartnerData/AbsenceDaysDetail/CCFRRWRVabsence09_16.txt')

TypeError: can't use a string pattern on a bytes-like object

In [25]:
csv.read()

Object `csv.read` not found.


In [87]:
df_IDs.

{'SchoolIDs':                         SchoolName        District SchoolID  DistrictID
 0      Coshocton Elementary School       Coshocton      S72         D12
 1          Roseville Middle School        Franklin      S73          D6
 2            South Lawn Elementary       Coshocton      S74         D12
 3              Nashport Elementary       TriValley       S1  D14       
 4               Morgan High School          Morgan      S10  D8        
 5     Thornville Elementary School        Northern      S11  D5        
 6         East Elementary (Morgan)          Morgan      S12  D8        
 7             Chieftain Elementary   Logan_Hocking      S13  D1        
 8        New Lexington High School   New Lexington      S14  D4        
 9    New Concord Elementary School  East Muskingum      S15  D7        
 10          Tri-Valley High School       TriValley      S16  D14       
 11                Union Elementary       Riverview      S17  D16       
 12              Dresden Elementary   